Comparing standard TD with Emphatic TD in a simple "conveyor belt" environment.

## Problem Setup

- Undiscounted case $\gamma = 0$ for all non-terminal states.
- Single feature with the same value for all states
- Interest uniform for all non-terminal states

In [42]:
import numpy as np
from collections import defaultdict
from copy import copy
from itertools import chain

import algorithms, environments,features, policy
from algorithms import EmphaticTD, TD
from environments.misc import ConveyorBelt
from features import Bias, Combination, Int2Unary, Wrapper
from policy import RandomPolicy

In [58]:
# Attribute logging class
class AttributeLogger:
    def __init__(self, obj, names):
        self._obj = obj
        self._names = [x for x in names]
        self.dct = {name: list() for name in names}
        
    def log(self):
        for name in self._names:
            value = getattr(self._obj, name)
            self.dct[name].append(copy(value))
            
    def __getitem__(self, key):
        return self.dct[key]

In [96]:
# For calculating errors in this domain
def mse(a, b):
    return np.mean((a - b)**2)

def geometric(val, n):
    """Geometric series: 1 + val + val^2 + ... + val^n"""
    return (1-val**n)/(1-val)

def triangle(n):
    return n*(n+1)/2

def pyramid(n):
    return n*(n+1)*(n+2)/6

In [3]:
def make_episodes(num_episodes, env, policy):
    return [make_episode(env, policy) for i in range(num_episodes)]
    

def make_episode(env, policy):
    env.reset()
    ret = []
    while not env.is_terminal():
        # Observe, take action, get next observation, and compute reward
        s  = env.observe()
        a  = policy(s)
        r  = env.do(a)
        sp = env.observe()

        # Append step to episode trajectory
        ret.append((s, a, r, sp))
    return ret

def apply_fa(episodes, phi_func):
    """Apply function approximation to a series of episodes."""
    ret = []
    for episode in episodes:
        tmp = []
        for step in episode[:-1]:
            s, a, r, sp = step
            fvec   = phi_func(s)
            fvec_p = phi_func(sp)
            tmp.append((fvec, a, r, fvec_p))
        
        # Account for final step of the episode
        s, a, r, sp = episode[-1]
        fvec   = phi_func(s)
        fvec_p = np.zeros(phi_func.length, dtype=np.float)
        tmp.append((fvec, a, r, fvec_p))
        ret.append(tmp)
    return ret

In [98]:
# Generate some episodes
random_seed = 101
num_episodes = 1000
num_states = 3

# Setup environment
env = ConveyorBelt(num_states, random_seed=random_seed)
env.reward = lambda s, a, sp: 1

# Set policy
pol = RandomPolicy(env, random_seed=random_seed)

# Get the raw episode data
raw_episodes = make_episodes(num_episodes, env, pol)

# Convert to features for function approximation
phi = Wrapper(Bias(), terminals=env.terminals)

# Apply function approximation
episodes = apply_fa(raw_episodes, phi)

## Expected Return

In [116]:
class ExpectedReturn:
    """Estimate expected returns (for tabular case)."""
    def __init__(self, gamma=1.0):
        self._gamma = gamma
        
    def __call__(self, episodes):
        value  = defaultdict(list)
        visits = defaultdict(int)
        for episode in episodes:
            ret = 0
            for step in reversed(episode):
                s, a, r, sp = step
                if isinstance(s, np.ndarray):
                    s = tuple(s)
                elif hasattr(s, '__iter__'):
                    s = tuple(s)
                ret = ret*self._gamma + r
                visits[s] += 1
                value[s].append(ret)
        return {s: sum(value[s])/visits[s] for s in value.keys()}
    
ER = ExpectedReturn()
exp_ret = ER(raw_episodes)
print(exp_ret)

{0: 3.0, 1: 2.0, 2: 1.0}


## Monte Carlo Value Approximation

In [114]:
class EveryVisitMC:
    def __init__(self, n, gamma=1.0):
        self.n = n
        self._gamma = gamma
        
    def __call__(self, episodes):
        value  = np.zeros(self.n)
        visits = np.zeros(self.n)
        for episode in episodes:
            ret = 0
            for step in reversed(episode):
                s, a, r, sp = step
                s = np.array(s)
                ret = ret*self._gamma + r
                visits += s
                value += s*ret
        return value/visits

In [115]:
MC = EveryVisitMC(phi.length, gamma=1.0)
MC(episodes)

array([ 2.])

## TD

In [109]:
# Specify the parameters for the episode
fixed_params = {'alpha': 0.01, 
                'gamma': 1.0, 
                'lmbda': 0.5}

# Parameters which are functions (e.g., of state)
param_funcs = {}

# Setup the agent
agent = TD(phi.length)

# Keep track of information over the episodes
log_names = ['theta', 'z']
logger = AttributeLogger(agent, log_names)

# Perform learning
for episode in episodes:
    agent.reset()
    for step in episode:
        s, a, r, sp = step
        
        # Parameters for update
        params = {}
        for name, func in param_funcs.items():
            params[k] = func(s)
        params.update(**fixed_params)
        
        # Update the agent
        agent.update(s, r, sp, params)
        
        # Record information
        logger.log()
        
print(agent.theta)

[ 2.40357138]


In [111]:
# Calculation for the asymptopic TD value
ns = 3
lm = 0.5

traces = [geometric(lm, n) for n in range(1, ns+1)]
approx = sum(traces)/(traces[-1])
print(approx)

2.4285714285714284


## Emphatic TD

In [68]:
# Specify the parameters for the episode
fixed_params = {'alpha': 0.01, 
                'gamma': 1.0, 
                'lmbda': 0.0, 
                'interest': 1.0,}

# Parameters which are functions (e.g., of state)
param_funcs = {}

# Setup the agent
agent = EmphaticTD(phi.length)

# Keep track of information over the episodes
log_names = ['theta', 'z']
logger = AttributeLogger(agent, log_names)

# Perform learning
for episode in episodes:
    agent.reset()
    for step in episode:
        s, a, r, sp = step
        
        # Parameters for update
        params = {}
        for name, func in param_funcs.items():
            params[k] = func(s)
        params.update(**fixed_params)
        
        # Update the agent
        agent.update(s, r, sp, params)
        
        # Record information
        logger.log()
        
print(agent.theta)

[ 2.44]


array([ 1.])